<a href="https://colab.research.google.com/github/ELMACHKOURIImane/PromptEngineering/blob/main/Phi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [ ]:
huggingface_dataset_name = "ELMACHKOURImane/DDD_Batch_2"
dataset = load_dataset(huggingface_dataset_name)

Generating train split:   0%|          | 0/2200 [00:00<?, ? examples/s]

In [ ]:
dataset['train']['message'][0]

'La blonde n\'avait pas l\'air d\'arrêter de dire qu\'il était maladroit et au moins il a eu son nom. Avec un serre-main ferme mais rapide, elle s\'en alla avec le reste de la foule qui se dirigeait vers la caserne. Que les femmes seront sûrement quelque chose, il s\'est pensé à lui-même et a écarté la partie sur l\'allemand. Victor savait que ce serait gênant comme c\'est le cas avec tout, tolérer l\'allemand serait une chose, mais se faire des amis avec l\'un il pouvait déjà voir que ce serait une décision impopulaire avec les autres. D\'après ce qu\'elle a dit et son langage corporel concernant ce fait, il pouvait déjà dire que la plupart des autres ressentaient la même chose qu\'elle.\nIl attendait la majorité de la foule pour aller avant de suivre, il ne savait jamais vraiment pourquoi il n\'aimait pas les foules il était sûr que cela avait quelque chose à voir avec être dangereux ou quelque chose. Une autre chose étrange qu\'il a remarquée était beaucoup d\'officiers et d\'autres

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import set_seed

#Set seed for reproducibility
seed = 42
set_seed(seed)

# Set seed for reproducibility
seed = 42
set_seed(seed)

# Assuming you have a DatasetDict named 'dataset' containing your data
index = 10  # Choose an index from your dataset

# Extract the game story context from the 'message' column
prompt = "je veut une histoire d'un lion avec un tiger "

# Construct the prompt for the model
formatted_prompt = f"I want a story game for the discription that i give you :\n{prompt}\n\Story:\n"

# Tokenize the prompt
inputs = tokenizer(formatted_prompt, return_tensors='pt', padding=True, truncation=True).to(original_model.device)

# Generate text using the model
output_sequences = original_model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=1000,  # Generate up to 100 new tokens
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,  # Enable sampling to get more diverse outputs
    temperature=0.7,  # Adjust temperature for sampling
    top_k=50,  # Consider the top 50 tokens for each step
    top_p=0.95  # Use nucleus sampling with cumulative probability of 0.95
)

# Decode the generated text
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

# Print the results
dash_line = '-' * 100
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'MODEL GENERATION:\n{generated_text}')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
I want a story game for the discription that i give you :
je veut une histoire d'un lion avec un tiger 
\Story:

----------------------------------------------------------------------------------------------------
MODEL GENERATION:
I want a story game for the discription that i give you :
je veut une histoire d'un lion avec un tiger 
\Story:

A lion and a tiger are best friends. One day, while exploring the jungle, they come across a mysterious cave. Curiosity gets the better of them, and they decide to enter the cave. Inside, they find a treasure chest filled with gold and jewels. Excitedly, they each take a handful of treasure and decide to split it evenly. But as they're about to leave, they hear a loud growl coming from behind them. They turn around to see a ferocious dragon guarding the treasure chest. The lion and tiger quickly realize that they can't defeat the drag